<a href="https://colab.research.google.com/github/aytekin827/TIL/blob/main/search%20algorithm/%EC%B5%9C%EB%8B%A8%EA%B2%BD%EB%A1%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 최단경로

다익스트라, 플로이드 워셜, 벨만 포드

* **다익스트라 알고리즘**  
 한 지점에서 다른 특정 지점까지의 최단경로를 구해야 하는 경우   
 O(v^2) : 간단한 다익스트라 알고리즘 시간복잡도  
 **O(ElogV)** : 복잡한 다익스트라 알고리즘 시간복잡도, **우선순위 큐** 사용

 [원리]
 1. 출발노드를 설정한다.
 2. 최단 거리 테이블을 초기화한다.
 3. 방문하지 않은 노드 중에서 최단 거리가 가장 짧은 노드를 선택한다.
 4. 해당 노드를 거쳐 다른 노드로 가는 비용을 계산하여 최단 거리 테이블을 갱신한다. => **그리디 개념**
 5. 위 과정에서 3번과 4번을 반복한다.     

 [조건]  
 음의 간선이 없을때

 [활용]  
 GPS소프트웨어 기본 알고리즘

 [참고]  
 힙을 이용한 다익스트라 알고리즘은 최소신장트리(MST, Minimum Spanning Tree)문제를 풀때 사용되는 Prim알고리즘에도 적용된다.  
 일단 이정도만 알아두자.

* **플로이드 워셜 알고리즘**  
 모든 지점에서 다른 도든 지점까지의 최단경로를 모두 구해야 하는 경우
 **O(n^3)** : 플로이드 워셜 알고리즘 시간복잡도
 
 [점화식]  
 **DP개념이 사용됨**
 $$D_{ab} = min(D_{ab}, D_{ak} + D_{kb}) $$
   
 [원리]  
 2차원 리스트를 이용해서 모든 가능한 연결 간선들을 표현 한다.  
 그리고 하나의 노드를 꺼내어 각각 가능한 모든 간선의 경우의 수를 고려하여 최단거리를 구하고 2차원 리스트 값을 갱신한다.
 



In [ ]:
# 간단한 다익스트라 알고리즘
# O(V^2) - N:노드의 개수

# import sys
# input = sys.stdin.readline
INF = int(1e9) # 무한을 의미하는 값

# 노드의 개수, 간선의 개수를 입력받기
n,m = map(int,input().split())
# 시작 노드 번호를 입력받기
start = int(input())
# 각 노드에 연결되어 있는 노드에 대한 정보를 담는 리스트를 만들기
graph = [[] for i in range(n+1)]
# 방문한 적이 있는지 체크하는 목적의 리스트를 만들기
visited = [False] * (n+1)
# 최단거리 테이블을 모두 무한으로 설정
distance = [INF] * (n+1)

# 모든 간선 정보를 입력받기
for _ in range(m):
    a,b,c = map(int,input().split()) # a에서 b로 가는 비용이 c라는 의미.
    graph[a].append((b,c))

# 방문하지 않은 노드 중에서, 가장 최단 거리가 짦은 노드의 번호를 변환
def get_smallest_node():
    min_value = INF
    index = 0
    for i in range(1,n+1):
        if distance[i] < min_value and not visited[i]:
            min_value = distance[i]
            index = i
    return index

def dijkstra(start):
    # 시작 노드에 대해서 초기화
    distance[start] = 0
    visited[start] = True
    for j in graph[start]:
        distance[j[0]] = j[1]
    # 시작 노드를 제외한 전체 n-1 개의 노드에 대해 반복
    for i in range(n-1):
        # 현재 최단 거리가 가장 짧은 노드를 꺼내서, 방문처리
        now = get_smallest_node()
        visited[now] = True
        # 현재 노드와 연결된 다른 노드를 확인
        for j in graph[now]:
            cost = distance[now] + j[1]
            # 현재 노드를 거쳐서 다른 노드로 이동하는 거리가 더 짧은 경우
            if cost < distance[j[0]]:
                distance[j[0]] = cost

# 다익스트라 알고리즘 수행
dijkstra(start)

# 모든 노드로 가기 위한 최단 거리를 출력
for i in range(1,n+1):
    # 도달할 수 없는 경우, 무한(INFINITY)라고 출력
    if distance[i] == INF:
        print("INFINITY")
    # 도달할 수 있는 경우 거리를 출력
    else:
        print(distance[i])

In [ ]:
# 개선된 다익스트라 알고리즘
# O(ElogV) 더 짧은 시간복잡도(E:간선의 개수, V:노드의 개수)
# 우선순위 큐를 이용해서

import heapq
# import sys
# input = sys.stdin.readline
INF = int(1e9)

# 노드의 개수, 간선의 개수 입력받기
n,m = map(int,input().split())
# 시작 노드 번호를 입력받기
start = int(input())
# 각 노드에 연결되어 있는 노드에 대한 정보를 담는 리스트 만들기
graph = [[] for i in range(n+1)]
# 최단 거리 테이블을 모두 무한으로 초기화
distance = [INF] * (n+1)

# 모든 간선 정보를 입력받기
for _ in range(m):
    a,b,c = map(int,input().split())
    graph[a].append((b,c))

def dijkstra(start):
    q = []
    # 시작 노드로 가기 위한 최단 경로는 0으로 설정하여, 큐에 삽입
    heapq.heappush(q,(0,start))
    distance[start] = 0
    while q: # 큐가 비어있지 않다면
        dist, now = heapq.heqppop(q)
        # 현재 노드가 이미 처리된 적이 있는 노드라면 무시
        if distance[now] < dist:
            continue
        # 현재 노드와 연결된 다른 인접한 노드들을 확인
        for i in graph[now]:
            cost = dist + i[1]
            # 현재 노드를 거쳐서, 다른 노드로 이동하는 거리가 더 짧은 경우
            if cost < distance[i[0]]:
                distance[i[0]] = cost
                heapq.heappush(q,(cost,i[0]))

# 다익스트라 알고리즘을 수행
dijkstra(start)

# 모든 노드로 가기 위한 최단거리 출력
for i in range(1,n+1):
    # 도달할 수 없는 경우, 무한(INFINITY)라고 출력
    if distance[o] == INF:
        print("INFINITY")
    else:
        print(distacne[i])


In [ ]:
# 플로이드 워셜 알고리즘 소스코드
# O(n^3) 시간복잡도
# 모든 지점에서 다른 모든 지점까지의 최단 경로를 모두 구해야하는 경우 -> 2차원 리스트 사용

INF = int(1e9) # 무한을 의미하는 값 = 10억

# 노드의 개수 및 간선의 개수 입력받기
n,m = map(int,input().split())
# 2차원 리스트(그래프 포현)를 만들고, 모든 값을 무한으로 초기화
graph = [[INF] * (n+1) for _ in range(n+1)]
# 자기 자신에서 자기 자신으로 가는 비용은 0으로 초기화
for a in range(1,n+1):
    for b in range(1,n+1):
        if a==b:
            graph[a][b] = 0

# 각 간선에 대한 정보를 입력받아, 그 값으로 초기화
for _ in range(m):
    # A에서 B로 가는 비용은 C라고 설정
    a,b,c = map(int,input().split())
    graph[a][b] = c

# 점화식에 따라 플로이드 워셜 알고리즘 수행
for k in range(1,n+1):
    for a in range(1,n+1):
        for b in range(1,n+1):
            # 도달할 수 없는 경우, 무한(INFINITY)라고 출력
            if graph[a][b] == INF:
                print("INFINITY")
            else:
                print(graph[a][b],end=' ')
    print()